<a href="https://colab.research.google.com/github/misbahsy/APMonitor-do/blob/master/ModelInitialization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Model Initialization Strategies

<p class='vspace'>Simulation is a first step after the model development to verify convergence, validate the model response to input changes, and manually adjust parameters to fit an expected response. This section demonstrates how to set up and initialize a dynamic model simulation. Options such as <a class='urllink' href='https://apmonitor.com/wiki/index.php/Main/OptionApmCsvRead' rel='nofollow'>CSV_READ</a> control how much information is read from a data (CSV) file.
</p>
<p class='vspace'>A first example shows how to use a scripting language such as MATLAB or Python to provide input values for a matrix of parameters.
</p>
<div class='vspace'></div><div><img src='http://apmonitor.com/do/uploads/Main/download.png' alt='' title='' /> <a class='urllink' href='http://apmonitor.com/do/uploads/Main/apmonitor_matrix.zip' rel='nofollow'>Initialize Parameter Matrix (MATLAB and Python)</a></div>

In [0]:
#APM code currently not functional

from apm import *
import numpy as np

A = np.random.random((3,4))
n = np.size(A,0) # rows
m = np.size(A,1) # columns

# write model.apm
fid = open('model.apm','w')
fid.write('Constants \n')
fid.write(' n ='+str(n)+' \n')
fid.write(' m ='+str(m)+' \n')
fid.write(' \n')
fid.write('Parameters \n')
fid.write(' p[1:n][1::m] \n')
fid.write(' \n')
fid.write('Variables \n')
fid.write(' x \n')
fid.write('Equations \n')
fid.write(' x=p[1][1] \n')
fid.close()

# write data.csv
fid = open('data.csv','w')
for i in range(n):
    for j in range(m):
        fid.write(' p['+str(i+1)+']['+str(j+1)+'], '+str(A[i,j])+' \n')
fid.close()

# load model, data file, and solve
s = 'https://byu.apmonitor.com'
a = 'matrix_write'
apm(s,a,'clear all')
apm_load(s,a,'model.apm')
csv_load(s,a,'data.csv')
apm(s,a,'solve')

# retrieve solution

<p class='vspace'>Although a problem may be written correctly, sometimes the solver fails to find a solution or requires excessive time to find a solution. Initialization strategies are critical in these situations to find a nearby solution that seeds the optimization problem with a starting point that allows convergence<sup>1</sup>.
</p>
<p class='vspace'>The following example is a demonstration of inserting different initial conditions or parameter values at points throughout the time horizon. A simulation solution is used to provide guess values for a subsequent simulation. The parameter <a class='urllink' href='https://apmonitor.com/wiki/index.php/Main/OptionApmCsvRead' rel='nofollow'>CSV_READ</a> can be set to <em>2</em> to provide the initial values for a calculated state. The default (<em>CSV_READ=1</em>) only updates the fixed values and skips the values that are calculated by the solver. Setting <a class='urllink' href='https://apmonitor.com/wiki/index.php/Main/OptionApmColdstart' rel='nofollow'>COLDSTART</a> &gt;= 1 also has the effect of using calculated values in the CSV file as initial guesses for the solver.
</p>
<div class='vspace'></div><div><img src='http://apmonitor.com/do/uploads/Main/download.png' alt='' title='' /> <a class='urllink' href='http://apmonitor.com/do/uploads/Main/apmonitor_initialize.zip' rel='nofollow'>Initialize for Dynamic Simulation</a></div>
<div class='vspace'></div><div><img width='550px' src='http://apmonitor.com/do/uploads/Main/apmonitor_initialize.png' alt='' title='' /></div>

In [0]:
#APM Code not functional

import numpy as np
from apm import *  # load APMonitor library

##################################################################
## Step #1 - Solve model with p = 1
##################################################################

## Step #1a - write data.csv
n = 31
time = np.linspace(0,3,n)
p = np.ones(31)
x = 2 * np.ones(31)

fid = open('data.csv','w')

## write time row
fid.write('time, ')
for i in range(n-1):
    fid.write(str(time[i]) +  ', ')
fid.write(str(time[n-1]) + '\n')

## write 'p' row (input parameter)
fid.write('p, ')
for i in range(n-1):
    fid.write(str(p[i]) +  ', ')
fid.write(str(p[n-1]) + '\n')

## write 'x' row (state variable initialization)
fid.write('x, ')
# imode: https://apmonitor.com/wiki/index.php/Main/Modes
# for imode=4-6, include all initialization values
# for imode=7-9, include only the initial condition for variables
imode = 7
if ((imode>=4) and (imode<=6)):
    for i in range(n-1):
        fid.write(str(x[i]) + ', ')
    fid.write(str(x[n-1]) + '\n')
else:
    fid.write(str(x[0]) + ', ')
    for i in range(1,n-1):
        fid.write('-, ')
    fid.write('-\n')

# close file
fid.close()

## Step 1b - Load and solve model
s = 'https://byu.apmonitor.com'
a = 'model_init'

apm(s,a,'clear all')
apm_load(s,a,'model.apm')
csv_load(s,a,'data.csv')

apm_option(s,a,'apm.time_shift',1)
apm_option(s,a,'apm.imode',imode)
output1 = apm(s,a,'solve')

## Step 1c - Retrieve results with solution.csv
solution1 = apm_sol(s,a)

##################################################################
## Step 2 - Solve again with prior solution for initialization and
##          p as a step from 0 to 2
##################################################################

## Change to imode = 4 and change p trajectory
p[0:5] = 0.0
p[5:n] = 2.0

## Step 2a - Write new row at the end of solution.csv
fname = 'solution_' + a + '.csv'
fid = open(fname,'a')  # append to file
fid.write('p, ')
for i in range(n-1):
    fid.write(str(p[i]) +  ', ')
fid.write(str(p[n-1]) + '\n')
# close file
fid.close()

## Step 2b - Reload csv file for initialization
apm(s,a,'clear csv')
csv_load(s,a,fname)

## Step 2c - Solve again but with new inputs
imode = 4
apm_option(s,a,'apm.time_shift',0)
apm_option(s,a,'apm.imode',imode)
output2 = apm(s,a,'solve')
print(output2)

## Step 2d - Retrieve results with solution.csv
solution2 = apm_sol(s,a)

##################################################################
## Step 3 - Create plots
##################################################################
import matplotlib.pyplot as plt

plt.figure(1)
plt.subplot(2,1,1)
plt.plot(solution1['time'],solution1['p'],'k-',linewidth=2)
plt.plot(solution2['time'],solution2['p'],'b--',linewidth=2)
plt.legend([r'$p_1$',r'$p_2$'])
plt.subplot(2,1,2)
plt.plot(solution1['time'],solution1['x'],'r--',linewidth=2)
plt.plot(solution2['time'],solution2['x'],'g:',linewidth=2)
plt.legend([r'$x_1$',r'$x_2$'])
plt.xlabel('time')
plt.show()


##Exercise
<p><strong>Objective:</strong> Simulate a highly nonlinear system, using initialization strategies to find a suitable approximation for a future parameter estimation exercise. Create a MATLAB or Python script to simulate and display the results. <em>Estimated Time: 2 hours</em>
</p>
<p class='vspace'>The spread of HIV in a patient is approximated with balance equations on (H)ealthy, (I)nfected, and (V)irus population counts<sup>2</sup>.
</p>
<div class='vspace'></div><pre> Initial Conditions
 H = healthy cells = 1,000,000
 I = infected cells = 0
 V = virus = 100
 LV = log virus = 2

 Equations
 dH/dt = kr<sub>1</sub> - kr<sub>2</sub> H - kr<sub>3</sub> H V
 dI/dt = kr<sub>3</sub> H V - kr<sub>4</sub> I
 dV/dt = -kr<sub>3</sub> H V - kr<sub>5</sub> V + kr<sub>6</sub> I
 LV = log<sub>10</sub>(V)
</pre><p class='vspace'>There are six parameters (kr<sub>1..6</sub>) in the model that provide the rates of cell death, infection spread, virus replication, and other processes that determine the spread of HIV in the body.
</p>
<div class='vspace'></div><pre> Parameters
 kr<sub>1</sub> = new healthy cells
 kr<sub>2</sub> = death rate of healthy cells
 kr<sub>3</sub> = healthy cells converting to infected cells
 kr<sub>4</sub> = death rate of infected cells
 kr<sub>5</sub> = death rate of virus
 kr<sub>6</sub> = production of virus by infected cells
</pre><p class='vspace'>The following data is provided from a virus count over the course of 15 years. Note that the virus count information is reported in log scale.
</p>
<div class='vspace'></div><div><img src='http://apmonitor.com/do/uploads/Main/download.png' alt='' title='' /> <a class='urllink' href='http://apmonitor.com/do/uploads/Main/data_hiv.zip' rel='nofollow'>HIV Data and Model Files</a></div>
<div class='vspace'></div><div><img src='http://apmonitor.com/do/uploads/Main/hiv_virus_count.png' alt='' title='' /></div>
<p class='vspace'>With guess values for parameters (kr<sub>1..6</sub>), approximately match the laboratory data for this patient. <a class='wikilink' href='http://apmonitor.com/do/index.php/Main/EstimatorObjective'>A subsequent section</a> introduces methods for parameter estimation by minimizing an objective function.
</p>

##Solution

<div><img src='http://apmonitor.com/do/uploads/Main/download.png' alt='' title='' /> <a class='urllink' href='http://apmonitor.com/do/uploads/Main/simulate_hiv.zip' rel='nofollow'>HIV Simulation in MATLAB and Python</a></div>

In [2]:
#@title
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/0Et07u336Bo?rel=0" frameborder="0" allowfullscreen></iframe>
